## Train a model locally or remote

In [ ]:
from sklearn import datasets, svm
digits = datasets.load_digits()
clf = svm.SVC(gamma=0.001, C=100.,probability=True)
clf.fit(digits.data[:-1], digits.target[:-1])
clf.predict(digits.data[-1:])

In [ ]:
!pip show scikit-learn

### Save model file

In [ ]:
!pip install joblib
from joblib import dump
dump(clf, 'model.joblib')

## Step 1 : Write a model transform script

#### Make sure you have a ...

- "load_model" function
    - input args are model path
    - returns loaded model object
    - model name is the same as what you saved the model file as (see above step)
<br><br>
- "predict" function
    - input args are the loaded model object and a payload
    - returns the result of model.predict
    - make sure you format it as a single (or multiple) string return inside a list for real time (for mini batch)
    - from a client, a list  or string or np.array that is sent for prediction is interpreted as bytes. Do what you have to for converting back to list or string or np.array
    - return the error for debugging


In [ ]:
%%writefile modelscript_sklearn.py
import sklearn
from joblib import load
import numpy as np
import os

#Return loaded model
def load_model(modelpath):
    print(modelpath)
    clf = load(os.path.join(modelpath,'model.joblib'))
    print("loaded")
    return clf

# return prediction based on loaded model (from the step above) and an input payload
def predict(model, payload):
    try:
        # locally, payload may come in as an np.ndarray
        if type(payload)==np.ndarray:
            out = [str(model.predict(np.frombuffer(payload).reshape((1,64))))]
        # in remote / container based deployment, payload comes in as a stream of bytes
        else:
            out = [str(model.predict(np.frombuffer(payload[0]['body']).reshape((1,64))))]
    except Exception as e:
        out = [type(payload),str(e)] #useful for debugging!
    
    return out

## Does this work locally? (not "_in a container locally_", but _actually_ in local)

In [ ]:
from modelscript_sklearn import *
model = load_model('.')

In [ ]:
predict(model,digits.data[-1:])[0]

### ok great! Now let's install ezsmdeploy

In [ ]:
!pip install ezsmdeploy[locust]

In [ ]:
import ezsmdeploy

#### If you have been running other inference containers in local mode, stop existing containers to avoid conflict

In [ ]:
!docker container stop $(docker container ls -aq) >/dev/null

## Deploy locally

In [ ]:
ez = ezsmdeploy.Deploy(model = ['model.joblib'], # if you intend to add models later, pass model as list, otherwise str
                  script = 'modelscript_sklearn.py',
                  requirements = ['pyarrow','scikit-learn==0.22.1','numpy','joblib'], #or pass in the path to requirements.txt
                  instance_type = 'local',
                  autoscale = True,
                  #framework = 'sklearn', # not required if you provide requirements
                  wait = True)

## Test containerized version locally

In [ ]:
out = ez.predictor.predict(digits.data[-1:].tobytes()).decode()
out

In [ ]:
!docker container stop $(docker container ls -aq) >/dev/null

## Deploy on SageMaker

In [ ]:
ezonsm = ezsmdeploy.Deploy(model = ['model.joblib','model.joblib'], # example of multimodel endpoint. 
                  script = 'modelscript_sklearn.py',
                  requirements = ['pyarrow','scikit-learn==0.22.1','numpy','joblib'])

In [ ]:
out = ezonsm.predictor.predict(digits.data[-1:].tobytes(),target_model='model1.tar.gz').decode() 
out

In [ ]:
ezonsm.test(input_data=digits.data[-1:].tobytes(), target_model='model1.tar.gz',usercount=20,hatchrate=10,timeoutsecs=10)

In [ ]:
import pandas as pd

pd.read_csv('src/locuststats_stats.csv')

In [ ]:
ezonsm.predictor.delete_endpoint()